## Check List 4 - PlanningPipeline
*\* ~~Strikethrough~~ means no-error but none-functional cases.*
* **4.1 PlanningScene**  
  - create_binder
    - actor: PlacePlane, Gripper2Tool, SweepTool
    - handle: Grasp2Point, PlacePoint, SweepPoint
  - create_subject
    - subject: CustomObject, SweepLineTask
  - initialize_state
* **4.2 MotionPlanner**  
  - MoveitPlanner
    - pick, place, sweep
  - EtaslPlanner
    - pick, place, ~~sweep~~
* **4.3 TaskPlanner**  
  - RRT
    - pick & place & sweep
      - single process
      - multi process

## set running directory to project source

In [1]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## 4.1 PlanningScene

##### initialize CombinedRobot and GeometryScene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None)   # create scene builder without detector for virtual scene
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,0,0), (0,0,0)), None)]
                     , connection_list=[False])
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.show_pose(crob.home_pose)

connection command:
indy0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries

In [3]:
from pkg.geometry.geometry import *

# add environments (fixed=True for non-movable geometries)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=np.random.rand(3)*0, color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
#                            rpy=np.random.rand(3)*np.pi/16, color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.5,-0.2,0),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="floor")
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (0.5,0.2,0), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="floor")
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,0), 
                          rpy=(0,0,0), color=(0.2,0.2,0.8,1), display=True, fixed=True, collision=False)
 
# add movable (fixed=False for movable geometries)
box1 = gscene.create_safe(GEOTYPE.BOX, "box1", "base_link", (0.05,0.05,0.05), (0.3,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.2,0.2,1), display=True, fixed=False, collision=True)

obstacle = gscene.create_safe(GEOTYPE.BOX, "obstacle", "base_link", (0.05,0.05,0.05), (0.5,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=False, collision=True)

##### create PlanningScene

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create_binder
- Binders (or Actors) are binding points where objects can be attached (or binded)
- Examples are 
  - PlacePlane: plane for object placement
  - Gripper2Tool: 2-finger gripper tool for grasp objects
  - SweepTool: action point to pass waypoints for sweep task

In [5]:
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool, SweepTool

#### create PlacePlane on geometry "floor" and "goal"
* when **point** is set to *None*, the entire upper surface of the geometry becomes valid binding area.
* when **point** is set, the specific point becomes the only valid binding point.

In [6]:
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane, point=None)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=(0,0,0.005))

#### add collision boundary for gripper base

In [7]:
# - set link_name="indy0_tcp" to attach the geometry to end-effector link
# - it can be labeled as fixed=True, as it is "fixed" on the indy0_tcp link
gripper =  gscene.create_safe(GEOTYPE.BOX, "gripper", link_name="indy0_tcp", 
                                dims=(0.02,0.1,0.1), center=(0,0,0.05), rpy=(0,0,0), 
                                color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True)

# add gripper fingers - By setting parent="gripper", the position of geometry can be set relative to the parent geometry
finger1 =  gscene.create_safe(GEOTYPE.BOX, "finger1", link_name="indy0_tcp",
                              dims=(0.04,0.02,0.1), center=(0,0.04,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")
finger2 =  gscene.create_safe(GEOTYPE.BOX, "finger2", link_name="indy0_tcp", 
                              dims=(0.04,0.02,0.1), center=(0,-0.04,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")

# create Gripper2Tool binder
# Gripper2Tool is a 2-finger gripper, which can rotate along z-direction.
# To align the z-direction with the 2 fingers, rotate by 90 degree along roll axis.
# The gripping point is (0,0,0.11) in local coordinate of "gripper" geometry
pscene.create_binder(bname="grip0", gname="gripper", _type=Gripper2Tool, point=(0,0,0.11), rpy=(-np.pi/2,0,0))

#### Add virtual (no-collision) sweep face. the point is 0.2 m away from the "indy0_tcp" link

In [8]:
# To match the z-direction with the target surface, the geometry is rotated 180 degrees in pitch-axis.
sweep_face =  gscene.create_safe(GEOTYPE.BOX, "sweep_face", link_name="indy0_tcp", 
                                dims=(0.05,0.05,0.001), center=(0,0,0.2), rpy=(0,np.pi,0), 
                                color=(0.2,0.2,0.8,0.1), display=True, fixed=True, collision=False)
                                 
# create SweepTool binder
pscene.create_binder(bname="sweep_face", gname="sweep_face", _type=SweepTool, point=(0,0,0), rpy=(0,0,0))

##### create_subject
* Subject describes the tasks in the planning scene.
* There are 2 categories in subject:
  1. Object: The object has grip points and placement points for pick&place task
  2. Task: The task is can be any non-physical task. Check SweepLineTask for example
* The subjects can be composed of multiple action points. Examples are:
  1. Grasp2Point: grasping point for 2-finger gripper. 
  2. PlacePoint: The point to place object.
  3. SweepPoint: A waypoint for SweepLineTask.
  * The above 3 action points inherit DirectePoint, for which the orientation is free along z-axis. 
  * If "point" parameter is not set, the entire upper surface is becomes valid action area.

#### create box object with grasping points along positive & negative y-direction and placement point in the bottom face

In [9]:
from pkg.planning.constraint.constraint_subject import BoxObject
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=BoxObject)

#### create sweep task with 2 waypoints

In [10]:
from pkg.planning.constraint.constraint_subject import SweepLineTask, SweepPoint
sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp1": SweepPoint("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepPoint("wp2", wp2, [0,0,0.005], [0,0,0])}
                             )
sweep.fix_direction = False

##### initialize_state
* initialize_state(robot_pose) updates robot pose and gets corresponding binding status of current scene.
* state.node of ('floor', 0) means the first subject (object) is placed on the floor and the second subject (sweep) has passed 0 waypoints

In [11]:
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

('floor', 0)


## 4.2 MotionPlanner

In [12]:
# motion test script
def motion_fn(from_state, to_node=None, to_state=None, try_count=10, **kwargs):
    assert to_node is not None or to_state is not None, "Either to_node or to_state should be specified"
    for _ in range(try_count):
        if to_node is not None:
            available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
            to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)

        pscene.set_object_state(from_state)
        gscene.show_pose(from_state.Q)
        subjects, sub_ok = pscene.get_changing_subjects(from_state, to_state)
        if len(subjects) == 1:
            show_action_point(pscene, 
                              btf=to_state.binding_state[subjects[0]], 
                              Q=from_state.Q)
        with gtimer.block("plan"):
            Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, **kwargs)
        if success:
            break
    if success:
        pscene.set_object_state(from_state)
        gscene.show_motion(Traj, period=0.01)
        end_state = pscene.rebind_all(binding_list, LastQ)
        gscene.clear_highlight()
    else:
        print("Solution not found. Please try again, try to find error if it keeps failing.")
        raise(RuntimeError("Motion plan failure"))
    return end_state

##### MoveitPlanner
* *get_available_binding_dict()* gets available binding states for each subject in a dictionary
* *sample_leaf_state samples()* target state with given available_binding_dict and target node
* *rebind_all()* updates binding state and returns the resultant state
* The motions tested in this section are:
  - pick: move the object to "gripper"
  - place: move the object to "goal"
  - sweep: 
    1) approach to waypoint 1
    2) sweep to waypoint 2
    3) return to home pose

In [13]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.utils.code_scraps import show_action_point
mplan = MoveitPlanner(pscene)
mp_kwargs = {}

gtimer = GlobalTimer.instance()
gtimer.reset()

#### Pick and place

In [14]:
pick_state = motion_fn(initial_state, to_node=("gripper", 0), **mp_kwargs)

In [15]:
place_state = motion_fn(pick_state, to_node=("goal", 0), **mp_kwargs)

#### Sweep

In [16]:
sweep_approached = motion_fn(place_state, to_node=("goal", 1), **mp_kwargs)

In [17]:
sweep_finished = motion_fn(sweep_approached, to_node=("goal", 2), **mp_kwargs)

#### homing (joint motion)

In [18]:
to_state = sweep_finished.copy(pscene)
to_state.Q = initial_state.Q
home_returned = motion_fn(sweep_finished, to_state=to_state, **mp_kwargs)

##### EtaslPlanner
* same usage as the MoveitPlanner
  - pick, place, ~~sweep~~
* eTaSl is not planner, but Model Predictive Controller-based trajectory generator. Thus, it shows poor obstacle avoidance and fails manytimes.
* if it fails, just try the whole **Pick and place** sequence multiple times

In [19]:
from pkg.planning.motion.etasl.etasl_planner import EtaslPlanner
from pkg.utils.code_scraps import show_action_point
mplan = EtaslPlanner(pscene)
mp_kwargs = dict(timeout=1, vel_conv=0, err_conv=1e-3)

gtimer = GlobalTimer.instance()
gtimer.reset()

#### Pick and place

In [20]:
pick_state = motion_fn(initial_state, to_node=("gripper", 0), try_count=50, **mp_kwargs)

In [21]:
place_state = motion_fn(pick_state, to_node=("goal", 0), try_count=100, **mp_kwargs)

eTaSL exception: optimization failed during execution
eTaSL exception: optimization failed during execution
eTaSL exception: optimization failed during execution
eTaSL exception: optimization failed during execution


#### homing (joint motion)

In [22]:
to_state = place_state.copy(pscene)
to_state.Q = initial_state.Q
home_returned = motion_fn(place_state, to_state=to_state, **mp_kwargs)

===================== plan joint manipulation =====================


## 4.3 TaskPlanner

##### create PlanningPipeline

In [23]:
from pkg.planning.pipeline import PlanningPipeline
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
ppline = PlanningPipeline(pscene)
mplan = MoveitPlanner(pscene)
ppline.set_motion_planner(mplan)

In [24]:
from pkg.planning.filtering.grasp_filter import GraspChecker
# from pkg.planning.filtering.reach_filter import ReachChecker
# from pkg.planning.filtering.latticized_filter import LatticedChecker
gcheck = GraspChecker(pscene)
mplan.motion_filters = [gcheck]
# mplan.motion_filters = [gcheck, ReachChecker(pscene), LatticedChecker(pscene, gcheck)]

In [25]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


##### RRT

In [26]:
from pkg.planning.task.rrt import TaskRRT
ppline.set_task_planner(TaskRRT(pscene))

   Use a production WSGI server instead.


- pick & place & sweep
  - single process
  - set display=True to check all intermediate plan segments

In [27]:
ppline.search(initial_state, goal_nodes=[("goal", 2)], verbose=True,
              timeout_loop=100, multiprocess=False, timeout=1, timeout_constrained=5,
              display=True)
schedules = ppline.tplan.find_schedules(at_home=True)
if len(schedules)==0:
    schedules = ppline.tplan.find_schedules(at_home=False)
schedules_sorted = ppline.tplan.sort_schedule(schedules)
ppline.play_schedule(ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0]), period=0.01)

 * Debug mode: off
try: 0 - ('floor', 0)->('gripper', 0)
Motion Filer Failure: GraspChecker
result: 0 - ('floor', 0)->('gripper', 0) = fail
try: 0 - ('floor', 0)->('gripper', 0)
Motion Filer Failure: GraspChecker
result: 0 - ('floor', 0)->('gripper', 0) = fail
try: 0 - ('floor', 0)->('gripper', 0)
Motion Filer Failure: GraspChecker
result: 0 - ('floor', 0)->('gripper', 0) = fail
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 0->1 (1.02/100.0 s, steps/err: 11(489.238977432 ms)/0.00182022529411)
try: 0 - ('floor', 0)->('gripper', 0)
Motion Filer Failure: GraspChecker
result: 0 - ('floor', 0)->('gripper', 0) = fail
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('gripper', 0) = success
branching: 0->2 (2.95/100.0 s, steps/err: 53(1756.85214996 ms)/0.00126087642813)
try: 0 - ('floor', 0)->('gripper', 0)
try transition moti

try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('floor', 0)->('gripper', 0) = fail
try: 6 - ('floor', 1)->('floor', 2)
try constrained motion
end
constrained motion tried: True
result: 6 - ('floor', 1)->('floor', 2) = success
branching: 6->16 (26.29/100.0 s, steps/err: 41(1474.28798676 ms)/0.341695695344)
try: 16 - ('floor', 2)->('gripper', 2)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 16 - ('floor', 2)->('gripper', 2) = fail
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('gripper', 0) = success
branching: 0->17 (28.98/100.0 s, steps/err: 45(1496.35004997 ms)/0.00176387615949)
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 0->18 (29.8/100.0 s, steps/err: 20(813.663005829 ms)/0.00156880943525)
try: 14 - ('floor', 1)->('floor', 2)
try constraine

- pick & place & sweep  
  - multi process

In [28]:
ppline.search(initial_state, goal_nodes=[("goal", 2)], verbose=True, 
              timeout_loop=100, multiprocess=True, timeout=1, timeout_constrained=5)
schedules = ppline.tplan.find_schedules(at_home=True)
if len(schedules)==0:
    schedules = ppline.tplan.find_schedules(at_home=False)
schedules_sorted = ppline.tplan.sort_schedule(schedules)
ppline.play_schedule(ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0]), period=0.01)

Use 36/36 agents
try: 0 - ('floor', 0)->('floor', 1)
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
Motion Filer Failure: GraspChecker
try: 0 - ('floor', 0)->('gripper', 0)
result: 0 - ('floor', 0)->('gripper', 0) = fail
Motion Filer Failure: GraspChecker
result: 0 - ('floor', 0)->('gripper', 0) = fail
try: 0 - ('floor', 0)->('gripper', 0)
try: 0 - ('floor', 0)->('floor', 1)
transition motion tried: True
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
try transition motion
result: 0 - ('floor', 0)->('floor', 1) = success
transition motion tried: True
branching: 0->1 (0.17/100.0 s, steps/err: 18(92.6201343536 ms)/0.00189479559712)
try transition motion
try: 0 - ('floor', 0)->('gripper', 0)
result: 0 - ('floor', 0)->('floor', 1) = success
transition motion tried: True
try transition motion
transition motion tried: True
branching: 0->2 (0.23/100.0 s,

result: 18 - ('floor', 2)->('gripper', 2) = fail
try: 0 - ('floor', 0)->('floor', 1)
try: 14 - ('gripper', 1)->('gripper', 2)
constrained motion tried: True
try transition motion
transition motion tried: True
try constrained motion
result: 2 - ('floor', 1)->('floor', 2) = success
try: 11 - ('gripper', 0)->('gripper', 1)
branching: 2->19 (0.67/100.0 s, steps/err: 41(332.216978073 ms)/1.2005344387)
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
result: 9 - ('floor', 0)->('gripper', 0) = success
branching: 0->20 (0.69/100.0 s, steps/err: 15(129.378080368 ms)/0.00169248249011)
transition motion tried: True
transition motion tried: True
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 9->21 (0.7/100.0 s, steps/err: 64(137.157917023 ms)/0.00111839492026)
branching: 0->22 (0.7/100.0 s, steps/err: 14(47.8670597076 ms)/0.00154228777814)
result: 9 - ('floor', 0)->('floor', 1) = success
branching: 9->24

result: 9 - ('floor', 0)->('floor', 1) = success
try constrained motion
try: 9 - ('floor', 0)->('floor', 1)
result: 10 - ('floor', 1)->('floor', 2) = success
try: 22 - ('floor', 1)->('floor', 2)
branching: 10->37 (1.03/100.0 s, steps/err: 41(379.503965378 ms)/0.636864698415)
try: 37 - ('floor', 2)->('gripper', 2)
transition motion tried: True
try constrained motion
branching: 9->38 (1.03/100.0 s, steps/err: 38(143.358945847 ms)/0.00168057260524)
try: 11 - ('gripper', 0)->('goal', 0)
try transition motion
try transition motion
try transition motion
try: 6 - ('gripper', 0)->('floor', 0)
transition motion tried: True
result: 13 - ('goal', 0)->('goal', 1) = success
try constrained motion
try: 39 - ('goal', 1)->('goal', 2)
try: 24 - ('floor', 1)->('floor', 2)
try transition motion
try: 9 - ('floor', 0)->('floor', 1)
result: 8 - ('gripper', 0)->('gripper', 1) = success
try constrained motion
try transition motion
try constrained motion
branching: 13->39 (1.06/100.0 s, steps/err: 70(223.60396

++ adding return motion to acquired answer ++
branching: 4->57 (1.37/100.0 s, steps/err: 41(515.258073807 ms)/1.33192904798)
Motion Filer Failure: GraspChecker
constrained motion tried: True
result: 56 - ('gripper', 2)->('goal', 2) = fail
try: 54 - ('gripper', 0)->('floor', 0)
try: 54 - ('gripper', 0)->('floor', 0)
try: 9 - ('floor', 0)->('floor', 1)
transition motion tried: True
try: 57 - ('floor', 2)->('gripper', 2)
try: 54 - ('gripper', 0)->('gripper', 1)
Motion Filer Failure: GraspChecker
result: 3 - ('floor', 1)->('floor', 2) = success
end
++ adding return motion to acquired answer ++
Goal reached
Motion Filer Failure: GraspChecker
transition motion tried: True
result: 54 - ('gripper', 0)->('floor', 0) = fail
try transition motion
try transition motion
result: 57 - ('floor', 2)->('gripper', 2) = fail
try: 14 - ('gripper', 1)->('gripper', 2)
result: 57 - ('floor', 2)->('gripper', 2) = fail
result: 47 - ('gripper', 2)->('goal', 2) = success
try transition motion
try: 21 - ('gripper'

Motion Filer Failure: GraspChecker
end
result: 77 - ('floor', 2)->('gripper', 2) = fail
Goal reached
result: 43 - ('gripper', 2)->('floor', 2) = success
Goal reached
result: 9 - ('floor', 0)->('floor', 1) = success
constrained motion tried: True
result: 13 - ('goal', 0)->('gripper', 0) = fail
transition motion tried: True
++ adding return motion to acquired answer ++
Goal reached
======================= terminated 29: first answer acquired ===============================
result: 25 - ('floor', 1)->('floor', 2) = success
======================= terminated 11: first answer acquired ===============================
result: 0 - ('floor', 0)->('floor', 1) = success
result: 36 - ('goal', 1)->('goal', 2) = success
try: 80 - ('floor', 2)->('gripper', 2)
Goal reached
branching: 43->80 (1.7/100.0 s, steps/err: 54(215.893030167 ms)/0.00123465589285)
branching: 36->79 (1.71/100.0 s, steps/err: 41(653.517007828 ms)/2.72788317524)
result: 29 - ('goal', 1)->('goal', 2) = success
======================

branching: 24->96 (1.78/100.0 s, steps/err: 41(711.564064026 ms)/2.79843702784)
result: 63 - ('goal', 0)->('goal', 1) = success
end
Goal reached
branching: 13->97 (1.79/100.0 s, steps/err: 72(281.378984451 ms)/0.00159198716812)
branching: 63->94 (1.8/100.0 s, steps/err: 30(274.484157562 ms)/0.00128101024562)
end
Goal reached
result: 41 - ('gripper', 1)->('gripper', 2) = success
constrained motion tried: True
======================= terminated 33: first answer acquired from other agent ===============================
constrained motion tried: True
Goal reached
result: 46 - ('goal', 1)->('goal', 2) = success
branching: 41->98 (1.81/100.0 s, steps/err: 41(517.285823822 ms)/2.57362517702)
======================= terminated 22: first answer acquired from other agent ===============================
branching: 46->101 (1.82/100.0 s, steps/err: 41(581.549167633 ms)/0.163474461153)
======================= terminated 28: first answer acquired ===============================
++ adding return moti

transition motion tried: False
transition motion tried: False
Motion Plan Failure
Motion Plan Failure
result: 6 - ('gripper', 0)->('floor', 0) = fail
result: 11 - ('gripper', 0)->('goal', 0) = fail
======================= terminated 0: first answer acquired from other agent ===============================
======================= terminated 19: first answer acquired from other agent ===============================
======================= terminated 12: first answer acquired ===============================
transition motion tried: False
Motion Plan Failure
result: 54 - ('gripper', 0)->('floor', 0) = fail
======================= terminated 34: first answer acquired from other agent ===============================
transition motion tried: False
Motion Plan Failure
result: 48 - ('gripper', 2)->('floor', 2) = fail
======================= terminated 18: first answer acquired from other agent ===============================
('floor', 0)->('gripper', 0)
('gripper', 0)->('gripper', 1)
('gripper'